In [24]:
#getting our own spotify data

In [25]:
#importing libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import time
import datetime
import os
import csv
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing


In [26]:
#Authentication - without user using Spotify's Web API and making a new spotify developer account 
client_credentials_manager = SpotifyClientCredentials(client_id="14ab08163a0649be865dc476a3076fd7", client_secret="c33964ff56e24f868580c3d3c488304c")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [27]:
#Getting user playlist
playlist_link = "https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=0067958267de4b47"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"]
              for x in sp.playlist_tracks(playlist_URI)["items"]]


In [28]:
#Acosticness confidence measure from 0.0 to 1.0 of whether the track is acoustic.
#Danceability measure describes how suitable a track is for dancing.
#duration_ms is the duration of the song track in milliseconds.
#Energy represents a perceptual measure of intensity and activity.
#Instrumentalness predicts whether a track contains vocals or not.
#Loudness of a track in decibels(dB).
#Liveness detects the presence of an audience in the recording.
#Speechiness detects the presence of spoken words in a track
#Time_signature is an estimated overall time signature of a track.
#Key the track is in. Integers map to pitches using standard Pitch Class notation.
#Valence measures from 0.0 to 1.0 describing the musical positiveness conveyed by a track.
#Target value describes the encoded value of 0 and 1. 0 means listener has not saved the song and 1 means listener have saved the song.
#Tempo is in beats per minute(BPM).
#Mode indicates the modality(major or minor) of the song.
#Song_title is the name of the song.
#Artist is the singer of the song.


In [29]:
#making a new csv file with th data and headers as the features of the audio features and also Song Name and Artist Name
song_name = []
artist_name = []
acousticness = []
danceability = []
duration_ms = []
energy = []
instrumentalness = []
loudness = []
liveness=[]
speechiness = []
time_signature = []
key = []
valence = []
tempo = []
mode = []
target = []
#get track name and artist name
for track_uri in track_uris:
    track = sp.track(track_uri)
    song_name.append(track["name"])
    artist_name.append(track["artists"][0]["name"])
    audio_features = sp.audio_features(track_uri)
    acousticness.append(audio_features[0]["acousticness"])
    danceability.append(audio_features[0]["danceability"])
    duration_ms.append(audio_features[0]["duration_ms"])
    energy.append(audio_features[0]["energy"])
    instrumentalness.append(audio_features[0]["instrumentalness"])
    loudness.append(audio_features[0]["loudness"])
    liveness.append(audio_features[0]["liveness"])
    speechiness.append(audio_features[0]["speechiness"])
    time_signature.append(audio_features[0]["time_signature"])
    key.append(audio_features[0]["key"])
    valence.append(audio_features[0]["valence"])
    tempo.append(audio_features[0]["tempo"])
    mode.append(audio_features[0]["mode"])
    target.append(1)
#make a csv file with the data
df = pd.DataFrame({'Song Name':song_name, 'Artist Name':artist_name, 'Acousticness':acousticness, 'Danceability':danceability, 'Duration_ms':duration_ms, 'Energy':energy, 'Instrumentalness':instrumentalness, 'Loudness':loudness,'Liveness':liveness ,'Speechiness':speechiness, 'Time_signature':time_signature, 'Key':key, 'Valence':valence, 'Tempo':tempo, 'Mode':mode, 'Target':target})
df.to_csv('audio_features.csv', index=False)


In [30]:
df = pd.read_csv("Audio_features.csv")
df.head()

,Song Name,Artist Name,Acousticness,Danceability,Duration_ms,Energy,Instrumentalness,Loudness,Liveness,Speechiness,Time_signature,Key,Valence,Tempo,Mode,Target
0,Tumbling Dice,The Rolling Stones,0.500000,0.615,226333,0.910,0.511000,-4.248,0.281,0.0347,4,11,0.770,111.013,1,1
1,Wish You Were Here,Pink Floyd,0.735000,0.481,334744,0.262,0.011400,-15.730,0.832,0.0414,4,7,0.375,122.861,1,1
2,Smells Like Teen Spirit,Nirvana,0.000025,0.502,301920,0.912,0.000173,-4.556,0.106,0.0564,4,1,0.720,116.761,1,1
3,Paradise City,Guns N' Roses,0.016900,0.273,405640,0.952,0.011100,-8.762,0.142,0.0843,4,11,0.472,100.271,1,1
4,Another One Bites The Dust - Remastered 2011,Queen,0.112000,0.933,214653,0.528,0.329000,-6.472,0.163,0.1620,4,5,0.756,109.975,0,1


In [31]:
from sklearn.preprocessing import MinMaxScaler
feature_cols = ['Acousticness', 'Danceability', 'Duration_ms', 'Energy',
                'Instrumentalness', 'Key', 'Liveness', 'Loudness', 'Mode',
                'Speechiness', 'Tempo', 'Time_signature', 'Valence', ]

scaler = MinMaxScaler()
normalized_df = scaler.fit_transform(df[feature_cols])

print(normalized_df[:2])


[[0.67112968 0.58431373 0.2782383  0.8918239  0.61124402 1.
  0.30857071 0.83541909 1.         0.02424242 0.29743689 1.
  0.78377173]
 [0.98657672 0.40915033 0.55272433 0.07672956 0.01363636 0.63636364
  1.         0.         1.         0.04189723 0.3857408  1.
  0.34351315]]


In [32]:
# Create a pandas series with song titles as indices and indices as series values
indices = pd.Series(df.index, index=df['Song Name']).drop_duplicates()

# Create cosine similarity matrix based on given matrix
cosine = cosine_similarity(normalized_df)


def generate_recommendation(song_title, model_type=cosine):
    """
    Purpose: Function for song recommendations 
    Inputs: song title and type of similarity model
    Output: Pandas series of recommended songs
    """
    # Get song indices
    index = indices[song_title]
    # Get list of songs for given songs
    score = list(enumerate(model_type[indices[song_title]]))
    # Sort the most similar songs
    similarity_score = sorted(score, key=lambda x: x[1], reverse=True)
    # Select the top-10 recommend songs
    similarity_score = similarity_score[1:11]
    top_songs_index = [i[0] for i in similarity_score]
    # Top 10 recommende songs
    top_songs = df['Song Name'].iloc[top_songs_index]
    return top_songs


In [33]:
#Print all the songs in the playlist
for i in range(len(song_name)):
    print(song_name[i],":",artist_name[i])


Tumbling Dice : The Rolling Stones
Wish You Were Here : Pink Floyd
Smells Like Teen Spirit : Nirvana
Paradise City : Guns N' Roses
Another One Bites The Dust - Remastered 2011 : Queen
Highway to Hell : AC/DC
I Love Rock 'N Roll : Joan Jett & The Blackhearts
Whole Lotta Love - 1990 Remaster : Led Zeppelin
Creep : Radiohead
Something In The Way : Nirvana
Dream On : Aerosmith
Livin' On A Prayer : Bon Jovi
Don't Stop Believin' : Journey
Seven Nation Army : The White Stripes
Hotel California - 2013 Remaster : Eagles
La Grange - 2005 Remaster : ZZ Top
Heroes - 2017 Remaster : David Bowie
Tom Sawyer : Rush
Baba O'Riley : The Who
Hold the Line : TOTO
Paranoid : Black Sabbath
Police & Thieves - Remastered : The Clash
Under the Bridge : Red Hot Chili Peppers
Back In Black : AC/DC
Everlong : Foo Fighters
(I Can't Get No) Satisfaction - Mono Version : The Rolling Stones
Paradise By the Dashboard Light : Meat Loaf
Dreams - 2004 Remaster : Fleetwood Mac
We Will Rock You - Remastered 2011 : Queen
Ent

KeyboardInterrupt: Interrupted by user

In [ ]:
song_value=input("Write the song name you want to recommend from this playlist")

In [ ]:
print("Recommended Songs:")
print(generate_recommendation(song_value,cosine).values)

Recommended Songs:
['Heat Waves' 'Party' 'X ÚLTIMA VEZ' 'Un Coco' 'Neverita'
 'Bam Bam (feat. Ed Sheeran)' 'Enséñame a Bailar' 'good 4 u' 'abcdefu'
 'El Apagón']
